In [1]:
! pip install datasets

# 1. Generate Predictions
Load the pre-trained model on Hugging Face and use it to translate the test set

In [1]:
from datasets import load_dataset

dataset_name = "PaulLerner/oscar_inclure"
dataset = load_dataset(dataset_name)

# Extract the test set
ds_test = dataset["test"]

README.md:   0%|          | 0.00/196 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

(…)-00000-of-00001-c5ad5bc51a93e0e9.parquet:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

(…)-00000-of-00001-8f62749bd30326df.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

(…)-00000-of-00001-1823ceca8818b4ca.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54930 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6866 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6867 [00:00<?, ? examples/s]

In [2]:
from transformers import pipeline
from tqdm.auto import tqdm

# Load the translation pipeline with your model
translator = pipeline("translation", model="PaulLerner/fabien.ne_barthez", device=0)  # Ensure correct model name

# Prepare inputs
'''
ds_test is test dataset.
ds_test["translation"] is a list of dictionary entries with translations
Entries look like: {"fr": "standard French sentence", "fri": "inclusive French translation"}
'''
# collects all French (standard) sentences from the test dataset that need to be translated
# The list test_sources now contains all "fr" sentences.
test_sources = [ex["fr"] for ex in ds_test["translation"]]

# Generate translations with progress bar (batching for efficiency)
batch_size = 16  # Instead of translating one sentence at a time, it translates 16 at once
predictions = [] # The translated text is stored in predictions, which will have all inclusive French translations.

#Loop through test_sources in chunks of 16 (from i to i+batch_size).
for i in tqdm(range(0, len(test_sources), batch_size), desc="Translating"):
    # Extracts a batch of 16 sentences from test_sources.
    batch = test_sources[i : i + batch_size]
    # Calls translator(batch), which translates all 16 sentences at once
    # max_length=512, truncation=True prevents errors for long sentences by limiting output to 512 tokens.
    translated_batch = translator(batch, max_length=1024, truncation=True) # Truncates overly long outputs instead of crashing.
    # Stores translated sentences
    predictions.extend([t["translation_text"] for t in translated_batch])
# At the end, predictions will contain your model’s translations from standard French (fr) to inclusive French (fri).
# In format {"translation_text": "inclusive French sentence"}'

# Print first 5 examples
for src, pred in zip(test_sources[:5], predictions[:5]):
    print(f"Source (Standard French): {src}")
    print(f"Translated (Inclusive French): {pred}")
    print("-" * 80)  # Separator for readability

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


Translating:   0%|          | 0/430 [00:00<?, ?it/s]

Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max

Source (Standard French): Nombre de participants : non déterminant

Translated (Inclusive French): Nombre de participant.e.s : non déterminant
--------------------------------------------------------------------------------
Source (Standard French): Le message est clair : ces organisations et personnalités sont accusés de complicité dans les attentats commis ces dernières semaines

Translated (Inclusive French): Le message est clair : ces organisations et personnalités sont accusé.e.s de complicité dans les attentats commis ces dernières semaines.
--------------------------------------------------------------------------------
Source (Standard French): Toutes les informations utiles sur la sécurité des données et les éventuels risques pour la sécurité, sur le type d’enregistrement des données, leur étendue et leur conservation, et sur les droits des clients, doivent être communiquées.

Translated (Inclusive French): Toutes les informations utiles sur la sécurité des données et les éven

# 2. Compute BLEU and ROUGE Scores
Get these eval metrics using Hugging Face's evaluate library

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [5]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c139fd36e0bcc9b26f258d1fc89c9c6e5ed5a674f5ea9f7c6e25ebe647f51b46
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
import evaluate

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Extract reference translations
references = [[ex["fri"]] for ex in dataset["test"]["translation"]]

# Compute BLEU
bleu_score = bleu.compute(predictions=predictions, references=references)

# Compute ROUGE
rouge_score = rouge.compute(predictions=predictions, references=[ref[0] for ref in references])

print("BLEU:", bleu_score)
print("ROUGE:", rouge_score)


BLEU: {'bleu': 0.7116924945454666, 'precisions': [0.9473012418786161, 0.9125261130253127, 0.8838885663884396, 0.8574016214219976], 'brevity_penalty': 0.7910675679057726, 'length_ratio': 0.8101286207700295, 'translation_length': 191162, 'reference_length': 235965}
ROUGE: {'rouge1': 0.9510704163720254, 'rouge2': 0.9121051535526614, 'rougeL': 0.9436475764715577, 'rougeLsum': 0.9437014940442461}


In [8]:
!pip install unbabel-comet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.6.1
    Uninstalling torchmetrics-1.6.1:
      Successfully uninstalled torchmetrics-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

# 3. Compute COMET Score
COMET requires a source, reference, and hypothesis (prediction)

Assesses fluency sound of the translations

In [9]:
from comet import download_model, load_from_checkpoint

# Load a pre-trained COMET model
comet_model_path = download_model("Unbabel/wmt20-comet-da")
comet_model = load_from_checkpoint(comet_model_path)

# Prepare data
comet_data = [{"src": src, "mt": pred, "ref": ref[0]} for src, pred, ref in zip(test_sources, predictions, references)]

# Compute COMET score
comet_score = comet_model.predict(comet_data)

print("COMET Score:", comet_score)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/437 [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
Predicting DataLoader 0: 100%|██████████| 430/430 [04:48<00:00,  1.49it/s]


COMET Score: Prediction([('scores', [1.3067328929901123, 1.2787508964538574, 1.1210273504257202, 1.2185697555541992, 0.9581047892570496, 1.2726397514343262, 1.2443926334381104, 1.2816131114959717, 1.2727022171020508, 1.1010332107543945, 1.2198913097381592, 1.2073644399642944, 1.0667401552200317, 1.1108708381652832, 1.18130624294281, 1.2619259357452393, 1.313117504119873, 1.2679591178894043, 1.0784369707107544, 1.3239741325378418, 0.7307291626930237, 0.7917196750640869, 1.2299706935882568, 1.2920634746551514, 1.01609468460083, 1.241133689880371, 1.3033146858215332, 1.0452489852905273, 0.41690731048583984, 1.3041783571243286, 1.1479618549346924, 1.3127903938293457, 1.2378685474395752, 0.5698124170303345, 1.240303635597229, 1.233577847480774, 1.2177330255508423, 1.1016035079956055, 1.3290789127349854, 1.0040758848190308, 1.1758944988250732, 0.13713699579238892, 1.2533365488052368, 1.2232766151428223, 1.165274739265442, 1.1725609302520752, 1.1538581848144531, 1.3062831163406372, 1.23175787